# Analyzing Campaign Finance Records for 2024 Gubernatorial Race of Candidate Patrick Morrisey (R)

This project is necessary for multiple reasons, as mentioned in the [README document](https://github.com/austindizzy/wvcfrs-parser/blob/master/README.md), including the ability to do quick data analysis on political campaigns in West Virginia. Let's use the structured data returned by this tool to do some of that quick data analysis on the campaign finance data of Patrick Morrisey (R) for WV Governor.

In [ ]:
import sqlite3
import matplotlib.pyplot as plt
import pandas as pd


conn = sqlite3.connect("./morrisey-2024.sqlite3")

df_over_250 = pd.read_sql_query("SELECT * FROM contributions_over_250", conn)
df_under_250 = pd.read_sql_query("SELECT * FROM contributions_under_250", conn)
df_all = pd.concat(
    [
        df_over_250[["date", "name", "election_type", "amount"]],
        df_under_250[["date", "name", "election_type", "amount"]],
    ]
)

df_over_250["date"] = pd.to_datetime(df_over_250["date"])
df_under_250["date"] = pd.to_datetime(df_under_250["date"])
df_all["date"] = pd.to_datetime(df_all["date"])
df_all["name"] = df_all["name"].apply(lambda x: x.upper().strip())

# if employer/occupation is "RETIRED RETIRED", replace with "RETIRED"
# same with "HOMEMAKER HOMEMAKER"
df_over_250["employer_occupation"] = df_over_250["employer_occupation"].str.replace(
    "RETIRED RETIRED", "RETIRED"
)

def create_layout(title, subtitle, geo=False):
    return dict(
        title=dict(text=f"{title}<br><sup>{subtitle}</sup>", font=dict(size=24)),
        annotations=[
            dict(
                text="Source: West Virginia Secretary of State | Credits: @AustinDizzy",
                showarrow=False,
                xref="paper",
                yref="paper",
                x=0.5,
                y=-0.1,
                font=dict(size=12),
            ),
            dict(
                text="Geographic data only available where a contributor has given more than $250 in a given election cycle per W.Va. Code §3-8-5(a)",
                showarrow=False,
                xref="paper",
                yref="paper",
                x=0.5,
                y=0,
                font=dict(size=10),
            ) if geo else {},
        ],
        margin=dict(l=0, r=0, t=100, b=100),
        height=800,
        width=1200,
    )

# Basic statistics
print(f"Total number of contributions: {len(df_all)}")
print(f"Total amount of contributions: ${df_all['amount'].sum():,.2f}")
print(f"Average contribution amount: ${df_all['amount'].mean():,.2f}")
print(f"Median contribution amount: ${df_all['amount'].median():,.2f}")
print(f"Largest contribution: ${df_all['amount'].max():,.2f}")
print(f"Smallest contribution: ${df_all['amount'].min():,.2f}")

In [ ]:
# Create cumulative sums
df_cumulative = df_all.groupby('date').agg({
    'amount': 'sum',
    'name': lambda x: x.nunique()
}).cumsum().reset_index()

# Create the plot
plt.figure(figsize=(12, 8))
fig, ax1 = plt.subplots(figsize=(12, 8))

# Plot cumulative amount
ax1.plot(df_cumulative['date'], df_cumulative['amount'], color='#1f77b4', linewidth=3)
ax1.set_ylabel('Total Amount Raised ($)', color='#1f77b4', fontsize=12)
ax1.tick_params(axis='y', labelcolor='#1f77b4')

# Create a twin axis for number of donors
ax2 = ax1.twinx()
ax2.plot(df_cumulative['date'], df_cumulative['name'], color='#ff7f0e', linewidth=3)
ax2.set_ylabel('Number of Unique Donors', color='#ff7f0e', fontsize=12)
ax2.tick_params(axis='y', labelcolor='#ff7f0e')

plt.title("Morrisey 2024 Campaign Contributions")
plt.xlabel("Date")
ax1.grid(True, linestyle='--', alpha=0.7)

# Format y-axis labels to use commas as thousand separators
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

# Add annotations for final values
last_date = df_cumulative['date'].iloc[-1]
last_amount = df_cumulative['amount'].iloc[-1]
last_donors = df_cumulative['name'].iloc[-1]

ax1.annotate(f'${last_amount:,.0f}',
             (last_date, last_amount),
             xytext=(10, 10), textcoords='offset points',
             color='#1f77b4', fontweight='bold')

ax2.annotate(f'{last_donors:,.0f} donors', 
             (last_date, last_donors),
             xytext=(10, -10), textcoords='offset points',
             color='#ff7f0e', fontweight='bold')


plt.figtext(0.5, -0.05, "Source: West Virginia Secretary of State | Credits: @AustinDizzy", 
            ha="center", fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
import plotly.graph_objects as go

# Interactive Time Series
daily_data = df_all.resample("D", on="date").agg({"amount": "sum", "name": "nunique"})

fig = go.Figure()
fig.add_trace(go.Scatter(x=daily_data.index, y=daily_data["amount"],
                         mode='lines', name='Total Amount'))
fig.add_trace(go.Scatter(x=daily_data.index, y=daily_data["name"],
                         mode='lines', name='Unique Contributors', yaxis="y2"))

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Total Amount ($)",
    yaxis2=dict(title="Unique Contributors (#)", overlaying="y", side="right")
)

fig.update_layout(create_layout("Morrisey 2024 Campaign Contributions", "Total Amount and Unique Contributors"))

fig.show()

In [ ]:
import seaborn as sns

# Heatmap of Weekly Contributions
weekly_data = df_all.resample("W", on="date")["amount"].sum().to_frame()
weekly_data["Week"] = weekly_data.index.isocalendar().week
weekly_data["Year"] = weekly_data.index.year
pivot_data = weekly_data.pivot(index="Year", columns="Week", values="amount")

plt.figure(figsize=(20, 8))
sns.heatmap(pivot_data, cmap="YlOrRd", annot=True, fmt=".0f")
plt.title("Weekly Contribution Heatmap")
plt.xlabel("Week of the Year")
plt.ylabel("Year")
plt.show()

# Bubble Chart of Top Contributors
top_contributors = df_all.groupby("name").agg({
    "amount": ["sum", "count"],
    "date": "min"
}).sort_values(("amount", "sum"), ascending=False).head(500)

plt.figure(figsize=(15, 10))
plt.scatter(top_contributors[("date", "min")], top_contributors[("amount", "sum")],
            s=top_contributors[("amount", "count")] * 10, alpha=0.5)
plt.title("Top 500 Contributors: Total Amount vs First Donation Date")
plt.xlabel("Date of First Donation")
plt.ylabel("Total Amount Donated ($)")
plt.show()

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
import re

def extract_state(addr: str) -> str | None:
    match = re.compile(r'\, ([A-Z]{2}) \d{5}\s*$', re.IGNORECASE).search(addr.strip())
    if match:
        return match.group(1).strip().upper()

def extract_zip(addr : str) -> str | None:
    match = re.search(r'(\d{5})\s*$', addr.strip())
    if match:
        return match.group(1).strip()

# Extract state and ZIP code from the address
df_over_250["state"] = df_over_250["address"].apply(extract_state)
df_over_250["zip_code"] = df_over_250["address"].apply(extract_zip)

# Aggregate data by state and ZIP code and unique names
state_data = df_over_250.groupby("state").agg(
    total_amount=("amount", "sum"),
    num_contributions=("name", "nunique"),
)
state_data.columns = ["total_amount", "num_contributions"]
state_data = state_data.reset_index()

print(state_data.sort_values("total_amount", ascending=False))

# Function to categorize total amount
def categorize_amount(amount):
    if amount > 1000000:
        return "Over $1M"
    elif amount > 100000:
        return "Over $100K"
    elif amount > 50000:
        return "Over $50K"
    elif amount > 25000:
        return "Over $25K"
    else:
        return "Over $0"

# Add category column to state_data
state_data['category'] = state_data['total_amount'].apply(categorize_amount)

category_order = ["Over $1M", "Over $100K", "Over $50K", "Over $25K", "Over $0"]
category_order.reverse()
category_counts = state_data['category'].value_counts()
category_labels = [f"{cat} ({category_counts[cat]})" for cat in category_order if cat in category_counts.index]

# Define color map
color_map = {
    "Over $1M": "#081d58",
    "Over $100K": "#225ea8",
    "Over $50K": "#41b6c4",
    "Over $25K": "#7fcdbb",
    "Over $0": "#c7e9b4"
}

# Create a new column with labels including counts
state_data['category_with_count'] = state_data['category'].map(
    dict(zip(category_counts.index, category_labels))
)

fig_state_amt = px.choropleth(
    state_data,
    locations="state",
    locationmode="USA-states",
    color="category_with_count",
    hover_name="state",
    hover_data=["total_amount", "num_contributions"],
    scope="usa",
    color_discrete_map={label: color_map[cat] for cat, label in zip(category_counts.index, category_labels)},
    category_orders={"category_with_count": category_labels},
)

fig_state_amt.update_layout(
    create_layout(
        "Total Contributions by State", "Morrisey 2024 Campaign for WV Governor",
        geo=True
    ),
    geo=dict(
        showlakes=False,
        showrivers=False,
    ),
    legend_title="Total Amount",
    legend=dict(
        traceorder="normal",
        itemsizing="constant",
    )
)

# Add state abbreviations
for state in state_data["state"]:
    fig_state_amt.add_annotation(
        x=state,
        y=state_data[state_data["state"] == state]["total_amount"].iloc[0],
        text=state,
        showarrow=False,
        font=dict(size=8, color="black"),
    )

fig_state_amt.show()

In [ ]:
import pandas as pd
import plotly.express as px

data = [
    dict(state="WV", transaction_total=484316.39, number_of_transactions=129),
    dict(state="WA", transaction_total=11447.76, number_of_transactions=13),
    dict(state="VA", transaction_total=97647.25, number_of_transactions=364),
    dict(state="TX", transaction_total=800.22, number_of_transactions=16),
    dict(state="TN", transaction_total=307.45, number_of_transactions=2),
    dict(state="SC", transaction_total=19419.96, number_of_transactions=8),
    dict(state="PA", transaction_total=4639.19, number_of_transactions=11),
    dict(state="OK", transaction_total=5000, number_of_transactions=1),
    dict(state="OH", transaction_total=2988308.59, number_of_transactions=29),
    dict(state="NY", transaction_total=9320.41, number_of_transactions=7),
    dict(state="NJ", transaction_total=135621.33, number_of_transactions=11),
    dict(state="MO", transaction_total=2487.26, number_of_transactions=2),
    dict(state="MN", transaction_total=562.99, number_of_transactions=2),
    dict(state="MI", transaction_total=58353.25, number_of_transactions=11),
    dict(state="ME", transaction_total=1485.5, number_of_transactions=3),
    dict(state="MD", transaction_total=1734.22, number_of_transactions=11),
    dict(state="MA", transaction_total=27850.63, number_of_transactions=14),
    dict(state="LA", transaction_total=28848.05, number_of_transactions=153),
    dict(state="KY", transaction_total=35920.94, number_of_transactions=2),
    dict(state="IL", transaction_total=8702.36, number_of_transactions=30),
    dict(state="IA", transaction_total=1629.93, number_of_transactions=2),
    dict(state="FL", transaction_total=608.34, number_of_transactions=5),
    dict(state="DC", transaction_total=1018.05, number_of_transactions=3),
    dict(state="CT", transaction_total=861.11, number_of_transactions=4),
    dict(state="CO", transaction_total=250365.56, number_of_transactions=6),
    dict(state="CA", transaction_total=28415.71, number_of_transactions=82),
    dict(state="AZ", transaction_total=267.49, number_of_transactions=3),
    dict(state="AR", transaction_total=75.81, number_of_transactions=3),
    dict(state="AL", transaction_total=55076.92, number_of_transactions=7),
]

# Create a DataFrame
exp_df = pd.DataFrame(data)

# Calculate total transactions
total_transactions = exp_df["transaction_total"].sum()

# Calculate percentage of total transactions for each state
exp_df['percentage'] = (exp_df['transaction_total'] / total_transactions) * 100

# Create a choropleth map
fig = px.choropleth(exp_df, 
    locations="state", 
    locationmode="USA-states", 
    color="percentage",
    hover_name="state", 
    scope="usa", 
    color_continuous_scale="Viridis",
)

# Update layout
fig.update_layout(
    create_layout(
        "% Share of Expenses by Vendor's State",
        "Morrisey 2024 Campaign for WV Governor",
    ),
    geo=dict(
        showlakes=False,
        showrivers=False,
    ),
)

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Assuming you have your data in a DataFrame called 'data'
df = pd.DataFrame(data)

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1('Campaign Spending Distribution by State'),
    html.H2('Morrisey 2024 Campaign for WV Governor'),
    
    dcc.RadioItems(
        id='metric-switch',
        options=[
            {'label': 'Transaction Total', 'value': 'transaction_total'},
            {'label': 'Number of Transactions', 'value': 'number_of_transactions'}
        ],
        value='transaction_total',
        labelStyle={'display': 'inline-block', 'marginRight': 10}
    ),
    
    dcc.Graph(id='campaign-charts')
])

@app.callback(
    Output('campaign-charts', 'figure'),
    [Input('metric-switch', 'value')]
)
def update_charts(selected_metric):
    # Calculate total for the selected metric
    total = df[selected_metric].sum()

    # Calculate percentage of total for each state
    df['percentage'] = (df[selected_metric] / total) * 100

    # Sort by percentage descending
    df_sorted = df.sort_values('percentage', ascending=False)

    # Separate WV and group others
    wv_data = df_sorted[df_sorted['state'] == 'WV'].iloc[0]
    other_states = df_sorted[df_sorted['state'] != 'WV']
    other_data = pd.DataFrame([{
        'state': 'Other States',
        selected_metric: other_states[selected_metric].sum(),
        'percentage': other_states['percentage'].sum()
    }])

    # Combine WV and Other States
    pie_data = pd.concat([pd.DataFrame([wv_data]), other_data])

    # Create subplots: pie chart and bar chart
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'xy'}]])

    wv_color = '#EAAA00'
    other_color = '#225fa9'

    # Add pie chart
    fig.add_trace(go.Pie(labels=pie_data['state'], values=pie_data['percentage'], 
                         textinfo='label+percent', marker_colors=[wv_color, other_color], hole=.3, name='', legend=None), 1, 1)

    # Add bar chart (top 5 states including WV)
    top_states = pd.concat([pd.DataFrame([wv_data]), other_states.head(4)])
    fig.add_trace(go.Bar(x=top_states['state'], y=top_states['percentage'], 
                         text=top_states['percentage'].apply(lambda x: f'{x:.1f}%'), 
                         textposition='auto', name='', legend=None, marker_color=[wv_color if state == 'WV' else other_color for state in top_states['state']]), 1, 2)

    # Update layout
    metric_name = "Total Spend [$]" if selected_metric == 'transaction_total' else "Number of Transactions"
    fig.update_layout(
        title=f"Campaign {metric_name} Distribution by State (in % of Total)",
        showlegend=False,
        margin=dict(l=50, r=10, t=100, b=100),
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import requests
import io
import os
import zipfile

# Load the GeoNames dataset for ZIP code information
# Credits: https://www.geonames.org/about.html
geonames_url = "https://download.geonames.org/export/zip/US.zip"
if not os.path.exists("US.txt"):
    print("Downloading GeoNames ZIP code data...")
    z = zipfile.ZipFile(io.BytesIO(requests.get(geonames_url).content))
    z.extractall()

geonames = pd.read_csv(
    "./US.txt",
    sep="\t",
    header=None,
    names=[
        "country_code",
        "postal_code",
        "place_name",
        "admin_name1",
        "admin_code1",
        "admin_name2",
        "admin_code2",
        "admin_name3",
        "admin_code3",
        "latitude",
        "longitude",
        "accuracy",
    ],
)

zip_dict = dict(
    zip(geonames["postal_code"], zip(geonames["latitude"], geonames["longitude"]))
)

def get_lat_lon(zip_code):
    return zip_dict.get(int(zip_code.strip()), (np.nan, np.nan))

zip_data = df_over_250.groupby("zip_code").agg(
    {"amount": "sum", "name": "nunique", "state": "first"}
)
zip_data.columns = ["total_amount", "num_contributions", "state"]
zip_data = zip_data.reset_index()

# Add lat and lon to zip_data
zip_data["lat"], zip_data["lon"] = zip(*zip_data["zip_code"].apply(get_lat_lon))
# Remove rows with missing lat/lon
zip_data = zip_data.dropna(subset=["lat", "lon"])

# Create ZIP code heat map using scatter_geo
fig_zip_heat = px.scatter_geo(
    zip_data,
    lat="lat",
    lon="lon",
    color="total_amount",
    size="total_amount",
    hover_name="zip_code",
    hover_data=["state", "num_contributions"],
    scope="usa",
    color_continuous_scale="jet",
)

fig_zip_heat.update_layout(
    create_layout(
        "Contribution Heat Map by ZIP Code", "Morrisey 2024 Campaign for WV Governor", geo=True
    ),
    coloraxis_colorbar=dict(title="Total Amount ($)"),
)

fig_zip_heat.show()

In [ ]:
import requests
import io
import os
import zipfile
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.express as px

# Load the GeoNames dataset for ZIP code information
# (Keep this part of the code as is)

# ... (previous code for loading GeoNames data)

# Read the GeoJSON file with zip code boundaries
gdf = gpd.read_file("zips/tl_2023_us_zcta520.shp")

# Merge zip_data with the GeoJSON data
gdf = gdf.merge(zip_data, left_on="ZCTA5CE20", right_on="zip_code", how="right")

print(gdf.head())

# Create choropleth map
fig_zip_choropleth = px.choropleth(
    gdf,
    geojson=gdf.geometry,
    locations=gdf.index,
    color="total_amount",
    hover_name="zip_code",
    hover_data=["state", "num_contributions"],
    scope="usa",
    color_continuous_scale="jet",
)

fig_zip_choropleth.update_geos(
    fitbounds="locations", 
    visible=False,
    center={"lat": 38.5, "lon": -80.9},  # Center on West Virginia
)

fig_zip_choropleth.update_layout(
    create_layout(
        "Contribution Choropleth Map by ZIP Code",
        "Morrisey 2024 Campaign for WV Governor",
        geo=True
    ),
    coloraxis_colorbar=dict(title="Total Amount ($)"),
)

fig_zip_choropleth.show()

In [ ]:
import geopandas as gpd
import plotly.express as px
from shapely.geometry import Point
import requests

# Load the GeoJSON data
geojson_url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
gdf = gpd.read_file(geojson_url)
counties = requests.get(geojson_url).json()

# Filter for West Virginia counties (FIPS codes for WV start with '54')
wv_counties = gdf[gdf['STATE'] == '54']

def get_fips_for_point(lat, lon):
    point = Point(lon, lat)
    contains = wv_counties.contains(point)
    if contains.any():
        fips = wv_counties[contains].iloc[0]['id']
        county = wv_counties[contains].iloc[0]['NAME']
        return fips, county
    else:
        return None, None

# Assuming zip_data is your original dataframe with contribution data
# Filter for West Virginia data
wv_data = zip_data[zip_data['state'] == 'WV'].copy()

wv_data['fips'], wv_data['county'] = zip(*wv_data.apply(lambda x: get_fips_for_point(x['lat'], x['lon']), axis=1))

# Aggregate data by county
wv_county_data = wv_data.groupby(['fips', 'county']).agg({
    'total_amount': 'sum', 
    'num_contributions': 'sum'
}).reset_index()

# Calculate average contribution per county
wv_county_data['avg_contribution'] = wv_county_data['total_amount'] / wv_county_data['num_contributions']

print(len(wv_county_data.sort_values('total_amount', ascending=False)))

# Create the choropleth map
fig = px.choropleth(
    wv_county_data,
    geojson=counties,
    locations='fips',
    color='total_amount',
    hover_name='county',
    hover_data={
        'total_amount': ':$,.2f',
        'num_contributions': ':,',
        'avg_contribution': ':$,.2f'
    },
    color_continuous_scale='Viridis',
    projection='mercator'
)
# Update the map layout for proper visualization
fig.update_geos(
    fitbounds="locations", 
    visible=False,
    center={"lat": 38.5, "lon": -80.9},  # Center on West Virginia
)

fig.update_layout(
    coloraxis_colorbar={
        'title': 'Total Amount ($)',
        'tickformat': '$,.0f'
    },
    height=800,  # Increase the height of the figure
    margin={"r":0,"t":50,"l":0,"b":0}
)

fig.update_layout(
    create_layout(
        'Total Contributions by County in West Virginia',
        'Morrisey 2024 Campaign for WV Governor', geo=True
    )
)

# Show the map
fig.show()